In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [36]:
df=pd.read_csv("/kaggle/input/gene-expression-in-gastric-cancer/expression profile(8863 genes).csv")

In [37]:
df.isnull().sum().sum()

0

In [38]:
df.dtypes

Unnamed: 0                   object
TCGA-BR-4279-01             float64
TCGA-VQ-A94T-01             float64
TCGA-BR-8589-01             float64
TCGA-BR-A4IV-01             float64
                             ...   
GTEX-P4QT-1526-SM-3NMCT     float64
TCGA-CG-5734-11             float64
GTEX-QDVJ-1426-SM-48U1Y     float64
GTEX-ZYVF-2726-SM-5GID4     float64
GTEX-12BJ1-1726-SM-5HL9B    float64
Length: 624, dtype: object

In [39]:
Y=df["Unnamed: 0"]
X=df.drop(columns=["Unnamed: 0"])

In [40]:
label_encoder=LabelEncoder()
label=label_encoder.fit_transform(Y)

In [41]:
X_train,X_test,Y_train,Y_test=train_test_split(X,label,test_size=0.2,random_state=42)

In [42]:
X_train.shape

(7090, 623)

In [43]:
X_train=X_train.values
X_train_reshaped = np.repeat(X_train[:, np.newaxis, :], 10, axis=1)
X_test=X_test.values
X_test_reshaped = np.repeat(X_test[:, np.newaxis, :], 10, axis=1)
X_test_reshaped.shape

(1773, 10, 623)

In [44]:
Y_train

array([6013, 8803, 5749, ..., 5390,  860, 7270])

In [45]:
# log_reg=LogisticRegression()
# log_reg.fit(X_train,Y_train)
# log_reg.score(X_test,Y_test)

In [46]:
# rand_reg=RandomForestRegressor()
# rand_reg.fit(X_train,Y_train)
# rand_reg.score(X_test,Y_test)

# # Deep learning

In [100]:
from keras.layers import Dense,Flatten,LSTM,SimpleRNN,GRU,Dropout,Input,BatchNormalization
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2,l1_l2
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

In [128]:
Y_train_reshape=np.expand_dims(Y_train,axis=-1)
Y_test_reshape=np.expand_dims(Y_test,axis=-1)

Y_train_reshape=to_categorical(Y_train_reshape)
Y_test_reshape=to_categorical(Y_test_reshape)
Y_train_reshape.shape

(7090, 8863)

In [102]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [109]:
print(np.unique(Y_train_reshape))


[0. 1.]


In [117]:
inputs=Input(shape=(10,623))
S=SimpleRNN(32,return_sequences=True,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)
S=LSTM(64,return_sequences=True,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)
S=GRU(128,return_sequences=True,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)
S=GRU(256,return_sequences=True,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)

S=GRU(556,return_sequences=False,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)

# S=GRU(256,return_sequences=True,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
# S=Dropout(0.5)(S)
# S=BatchNormalization()(S)
# S=GRU(128,return_sequences=True,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
# S=Dropout(0.5)(S)
# S=BatchNormalization()(S)
# S=LSTM(64,return_sequences=True,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
# S=Dropout(0.5)(S)
# S=BatchNormalization()(S)
# S=SimpleRNN(32,return_sequences=False,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
# S=Dropout(0.5)(S)
# S=BatchNormalization()(S)
S=Flatten()(S)
outputs=Dense(8863)(S)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss="mse",metrics=["mae"])
model.summary()


Model: "functional_39"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_22 (InputLayer)     │ (None, 10, 623)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_118 (SimpleRNN)      │ (None, 10, 32)         │        20,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_148 (Dropout)           │ (None, 10, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_147         │ (None, 10, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 10, 64)         │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_149 (Dropout)           │ (None, 10, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_148         │ (None, 10, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_23 (GRU)                    │ (None, 10, 128)        │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_150 (Dropout)           │ (None, 10, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_149         │ (None, 10, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_24 (GRU)                    │ (None, 10, 256)        │       296,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_151 (Dropout)           │ (None, 10, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_150         │ (None, 10, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_25 (GRU)                    │ (None, 556)            │     1,357,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_152 (Dropout)           │ (None, 556)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_151         │ (None, 556)            │         2,224 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_19 (Flatten)            │ (None, 556)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 8863)           │     4,936,691 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,715,355 (25.62 MB)

 Trainable params: 6,713,283 (25.61 MB)

 Non-trainable params: 2,072 (8.09 KB)

In [119]:
model.fit(X_train_reshaped,Y_train_reshape,epochs=10,validation_split=0.2,batch_size=32,callbacks=[early_stopping])

Epoch 1/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 28s 155ms/step - loss: 1.3353e-04 - mae: 0.0016 - val_loss: 1.1459e-04 - val_mae: 8.4722e-04
Epoch 2/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 41s 156ms/step - loss: 1.3496e-04 - mae: 0.0017 - val_loss: 1.1473e-04 - val_mae: 8.7587e-04
Epoch 3/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 28s 156ms/step - loss: 1.3551e-04 - mae: 0.0017 - val_loss: 1.1467e-04 - val_mae: 8.0030e-04
Epoch 4/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 41s 155ms/step - loss: 1.3582e-04 - mae: 0.0017 - val_loss: 1.1437e-04 - val_mae: 7.7719e-04
Epoch 5/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 28s 157ms/step - loss: 1.3640e-04 - mae: 0.0017 - val_loss: 1.1620e-04 - val_mae: 9.6147e-04
Epoch 5: early stopping


In [125]:
loss,mae=model.evaluate(X_test_reshaped[:,0:8864],Y_test_reshape[:,0:8864])

InvalidArgumentError: Graph execution error:

Detected at node compile_loss/mse/sub defined at (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code

  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/opt/conda/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/opt/conda/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/opt/conda/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/opt/conda/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 545, in dispatc